In [97]:
print("🚀 Starting Data Transformation Pipeline")
print("=" * 50)

🚀 Starting Data Transformation Pipeline


# Data Transformation Pipeline for MLOps Using SageMaker Jupyter Notebooks
`This notebook demonstrates key data transformation techniques commonly used in machine learning pipelines. It follows MLOps best practices for data preprocessing and feature engineering using AWS SageMaker JupyterLab.`

## 1. Environment Setup and Configuration
### 📦 Step 1: Setup Environment

In [98]:
# 📦 Step 1: Setup Environment
import sagemaker
import boto3
import pandas as pd
import os
from sagemaker.processing import ProcessingInput, ProcessingOutput, ScriptProcessor
import json
import numpy as np
from datetime import datetime

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

print(f"SageMaker Role: {role}")
print(f"Default Bucket: {bucket}")

SageMaker Role: arn:aws:iam::891377300841:role/cfst-4286-a3bb98dd86788b99c4-SageMakerExecutionRole-cN71NiG3F0e1
Default Bucket: sagemaker-us-east-1-891377300841


### ⚙️ Step 2: Data Generation
Creating a realistic dataset that simulates common data quality challenges found in production environments.

In [99]:
import pandas as pd
import numpy as np
import json
import random
from datetime import datetime, timedelta
import os

# Set random seed for reproducibility
np.random.seed(42)

# Number of records
num_records = 20000

# Generate random data
data = {
    "id": np.arange(1, num_records + 1),
    "name": [f"Name_{i}" for i in np.random.randint(1, 1000, num_records)],
    "age": np.random.randint(18, 80, num_records),
    "salary": np.random.choice([50000, 60000, 70000, None], num_records),
    "hire_date": [
        (datetime.now() - timedelta(days=random.randint(0, 3650))).strftime("%Y-%m-%d")
        if random.random() > 0.1 else None
        for _ in range(num_records)
    ],
    "profile": [
        json.dumps({
            "address": f"Street {random.randint(1, 100)}, City {random.randint(1, 50)}",
            "phone": f"{random.randint(1000000000, 9999999999)}",
            "email": f"email_{random.randint(1, 1000)}@example.com"
        })
        if random.random() > 0.1 else None
        for _ in range(num_records)
    ],
    "department": np.random.choice(["HR", "IT", "Finance", "Marketing", None], num_records),
    "bonus": [None if random.random() > 0.9 else random.randint(1000, 10000) for _ in range(num_records)]
}

# Create DataFrame
df = pd.DataFrame(data)

# Introduce some NaN values randomly
df.loc[np.random.choice(df.index, size=int(num_records * 0.05), replace=False), "age"] = np.nan
df.loc[np.random.choice(df.index, size=int(num_records * 0.1), replace=False), "salary"] = np.nan

# Ensure 'data' folder exists
os.makedirs("data", exist_ok=True)

# Save to CSV
df.to_csv("data/mock_data.csv", index=False)
print("Dataset created and uploaded to data/mock_data.csv")

Dataset created and uploaded to data/mock_data.csv


### ⚙️ Step 3: Upload Source Data to S3
Upload the source CSV dataset to input location in S3 (default bucket)  

In [100]:
s3 = boto3.resource('s3')
s3.meta.client.upload_file('data/mock_data.csv', bucket, 'input/mock_data.csv')
print(f"Dataset 'mock_data.csv' uploaded to: s3://{bucket}/input/mock_data.csv")

Dataset 'mock_data.csv' uploaded to: s3://sagemaker-us-east-1-891377300841/input/mock_data.csv


## 2. Data Exploration  
Load the raw dataset and perform initial data profiling. 
This step is crucial for understanding data quality and structure. 

### Step 1: Load the CSV File from S3 into the DataFrame

In [101]:
try:
    df = pd.read_csv(f's3://{bucket}/input/mock_data.csv')
    print(f"✅ Dataset loaded successfully!")
    print(f"📏 Dataset shape: {df.shape}")
except FileNotFoundError:
    print("❌ Error: mock_data.csv not found. Please run create_dataset.py first.")
    exit()

✅ Dataset loaded successfully!
📏 Dataset shape: (20000, 8)


### Step 2: Analyse the Data  
Perform comprehensive data analysis to understand:
- Data types and memory usage
- Missing values pattern
- Statistical distribution
- Unique values and categories

In [102]:
# Display the first 5 rows from the loaded DataFrame
print("\n📋 First 5 rows:")
df.head()


📋 First 5 rows:


,id,name,age,salary,hire_date,profile,department,bonus
0,1,Name_103,77.0,60000.0,2023-05-11,"{""address"": ""Street 51, City 12"", ""phone"": ""73...",Marketing,8767.0
1,2,Name_436,62.0,50000.0,2021-04-22,"{""address"": ""Street 12, City 6"", ""phone"": ""963...",Marketing,7553.0
2,3,Name_861,61.0,60000.0,2022-04-18,NaN,HR,6796.0
3,4,Name_271,36.0,70000.0,NaN,"{""address"": ""Street 17, City 5"", ""phone"": ""842...",NaN,8712.0
4,5,Name_107,78.0,60000.0,2017-06-17,"{""address"": ""Street 98, City 37"", ""phone"": ""81...",IT,3340.0


In [103]:
# Get the summary of the DataFrame
print("\n📊 Data Types & Non-Null Counts:\n")
df.info()


📊 Data Types & Non-Null Counts:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          20000 non-null  int64  
 1   name        20000 non-null  object 
 2   age         19000 non-null  float64
 3   salary      13519 non-null  float64
 4   hire_date   17934 non-null  object 
 5   profile     17967 non-null  object 
 6   department  16003 non-null  object 
 7   bonus       18001 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 1.2+ MB


In [104]:
# Check for duplicate rows
duplicates = df.duplicated().sum()
print(f"\n🔄 Duplicate rows: {duplicates}")


🔄 Duplicate rows: 0


In [105]:
# Check unique values in the department column
df['department'].unique()

array(['Marketing', 'HR', nan, 'IT', 'Finance'], dtype=object)

In [106]:
# View statistical summary for numeric coloums
print("\n📈 Statistical Summary:")
df.describe(include='all')


📈 Statistical Summary:


,id,name,age,salary,hire_date,profile,department,bonus
count,20000.000000,20000,19000.000000,13519.000000,17934,17967,16003,18001.000000
unique,NaN,999,NaN,NaN,3634,17967,4,NaN
top,NaN,Name_825,NaN,NaN,2020-09-14,"{""address"": ""Street 55, City 12"", ""phone"": ""76...",IT,NaN
freq,NaN,37,NaN,NaN,14,1,4058,NaN
mean,10000.500000,NaN,48.444684,59962.275316,NaN,NaN,NaN,5517.326149
std,5773.647028,NaN,17.892848,8200.588356,NaN,NaN,NaN,2608.397456
min,1.000000,NaN,18.000000,50000.000000,NaN,NaN,NaN,1000.000000
25%,5000.750000,NaN,33.000000,50000.000000,NaN,NaN,NaN,3247.000000
50%,10000.500000,NaN,48.000000,60000.000000,NaN,NaN,NaN,5531.000000
75%,15000.250000,NaN,64.000000,70000.000000,NaN,NaN,NaN,7783.000000


In [107]:
# Check for missing values
print("\n❓ Missing Values Analysis:\n")
df.isnull().sum()


❓ Missing Values Analysis:



id               0
name             0
age           1000
salary        6481
hire_date     2066
profile       2033
department    3997
bonus         1999
dtype: int64

## 🧹 3. Data Cleaning & Quality Improvement

### Step 1: Handle Missing values of age, and salary
Handle missing values in age and salary columns using appropriate strategies:
- For age: Use median (robust to outliers)
- For salary: Use median (robust to outliers)

In [108]:
# Analyze missing patterns
print("\n📊 Missing Value Patterns:")
print("Missing Age values:")
print(df[df['age'].isnull()][['age', 'salary', 'department']])


📊 Missing Value Patterns:
Missing Age values:
       age   salary department
44     NaN  60000.0  Marketing
115    NaN  60000.0         IT
127    NaN      NaN  Marketing
147    NaN  60000.0         HR
164    NaN  70000.0         IT
...    ...      ...        ...
19872  NaN  60000.0         HR
19921  NaN      NaN         HR
19940  NaN  70000.0        NaN
19997  NaN  60000.0         IT
19998  NaN  60000.0  Marketing

[1000 rows x 3 columns]


In [109]:
print("Missing Salary values")
print(df[df['salary'].isnull()][['age', 'salary', 'department']])

Missing Salary values
        age  salary department
5      35.0     NaN         IT
11     61.0     NaN         IT
13     46.0     NaN        NaN
14     48.0     NaN         IT
15     61.0     NaN         HR
...     ...     ...        ...
19984  71.0     NaN        NaN
19988  72.0     NaN  Marketing
19992  60.0     NaN        NaN
19993  76.0     NaN  Marketing
19999  47.0     NaN        NaN

[6481 rows x 3 columns]


In [110]:
# Get the median values for age, and salary
age_median = df['age'].median()
salary_median = df['salary'].median()
print("Age Median", age_median)
print("Salary Median", salary_median)

Age Median 48.0
Salary Median 60000.0


In [111]:
# Fill missing values of age with age_median
df['age'] = df['age'].fillna(age_median)
# Fill missing values of salary with salary_median
df['salary'] = df['salary'].fillna(salary_median)

#### Age & Salary columns missing values are filled with the respective median

In [112]:
# Verify the Age & Salary data
df.head()
# Check for missing values
print("Missing values in each column")
df.isnull().sum()

Missing values in each column


id               0
name             0
age              0
salary           0
hire_date     2066
profile       2033
department    3997
bonus         1999
dtype: int64

### Step 2: Handle Missing values of Department
Handle missing values in categorical columns:
- For department: Use 'Unknown' category
- This preserves the information that the department was missing

In [113]:
print("Print the missing values for Department\n")
print("Missing Department Missing values")
print(df[df['department'].isnull()][['age', 'salary', 'department']])

Print the missing values for Department

Missing Department Missing values
        age   salary department
3      36.0  70000.0        NaN
13     46.0  60000.0        NaN
49     34.0  50000.0        NaN
53     33.0  60000.0        NaN
57     28.0  70000.0        NaN
...     ...      ...        ...
19973  50.0  60000.0        NaN
19975  29.0  60000.0        NaN
19984  71.0  60000.0        NaN
19992  60.0  60000.0        NaN
19999  47.0  60000.0        NaN

[3997 rows x 3 columns]


In [114]:
# Fill the missing values in department with 'Unknown'
df['department'] = df['department'].fillna('Unknown')

#### Department column missing values are filled with the respective median

In [115]:
# Verify the Age & Salary data
df.head()
# Check for missing values
print("Missing values in each column")
print(df.isnull().sum())
# Check unique values in the department column
df['department'].unique()

Missing values in each column
id               0
name             0
age              0
salary           0
hire_date     2066
profile       2033
department       0
bonus         1999
dtype: int64


array(['Marketing', 'HR', 'Unknown', 'IT', 'Finance'], dtype=object)

### Step 3: Parse and Extract Profile Information
Devide Profile Column into 3 different columns i.e., Address, Phone, Email   

Parse JSON profile data and extract structured information:
- Extract address, phone, and email into separate columns
- Handle malformed JSON gracefully
- Maintain data integrity during extraction

In [116]:
print("Top rows from profile column \n")
print(df['profile'].head())

# Find the first non-null value in the column
profile_first_value = df['profile'].dropna().iloc[0]
# Print its type
print("\nProfile column values current data type")
print(type(profile_first_value))

# If your 'profile' column already contains Python dictionaries, not JSON strings.
# You do not need to parse it with json.loads(). The data is ready to be used directly.

# Convert profile JSON strings into dictionaries
df['profile'] = df['profile'].apply(lambda x: json.loads(x) if pd.notnull(x) else {})

Top rows from profile column 

0    {"address": "Street 51, City 12", "phone": "73...
1    {"address": "Street 12, City 6", "phone": "963...
2                                                  NaN
3    {"address": "Street 17, City 5", "phone": "842...
4    {"address": "Street 98, City 37", "phone": "81...
Name: profile, dtype: object

Profile column values current data type
<class 'str'>


In [117]:
# Extract Address Field
print("Extract Address Field....\n")
# Create new 'address' column by extracting from 'profile' dictionaries
df['address'] = df['profile'].apply(lambda x: x.get('address', None))  # Returns None if no address key

print("Top rows from profile column \n")
print(df['profile'].head())
print("\nTop rows from newly created address column \n")
print(df['address'].head())


Extract Address Field....

Top rows from profile column 

0    {'address': 'Street 51, City 12', 'phone': '73...
1    {'address': 'Street 12, City 6', 'phone': '963...
2                                                   {}
3    {'address': 'Street 17, City 5', 'phone': '842...
4    {'address': 'Street 98, City 37', 'phone': '81...
Name: profile, dtype: object

Top rows from newly created address column 

0    Street 51, City 12
1     Street 12, City 6
2                  None
3     Street 17, City 5
4    Street 98, City 37
Name: address, dtype: object


In [118]:
# Extract Phone Field
print("Extract Phone Field....\n")
# Create new 'phone' column by extracting from 'profile' dictionaries
df['phone'] = df['profile'].apply(lambda x: x.get('phone', None))  # Returns None if no address key

print("Top rows from profile column \n")
print(df['profile'].head())
print("\nTop rows from newly created phone column \n")
print(df['phone'].head())


Extract Phone Field....

Top rows from profile column 

0    {'address': 'Street 51, City 12', 'phone': '73...
1    {'address': 'Street 12, City 6', 'phone': '963...
2                                                   {}
3    {'address': 'Street 17, City 5', 'phone': '842...
4    {'address': 'Street 98, City 37', 'phone': '81...
Name: profile, dtype: object

Top rows from newly created phone column 

0    7323475298
1    9633478479
2          None
3    8423075763
4    8158889415
Name: phone, dtype: object


In [119]:
# Extract Email Field
print("Extract Email Field....\n")
# Create new 'email' column by extracting from 'profile' dictionaries
df['email'] = df['profile'].apply(lambda x: x.get('email', None))  # Returns None if no address key

print("Top rows from profile column \n")
print(df['profile'].head())
print("\nTop rows from newly created email column \n")
print(df['email'].head())

print(f"\n✅ Profile fields extracted:")

Extract Email Field....

Top rows from profile column 

0    {'address': 'Street 51, City 12', 'phone': '73...
1    {'address': 'Street 12, City 6', 'phone': '963...
2                                                   {}
3    {'address': 'Street 17, City 5', 'phone': '842...
4    {'address': 'Street 98, City 37', 'phone': '81...
Name: profile, dtype: object

Top rows from newly created email column 

0    email_140@example.com
1     email_38@example.com
2                     None
3    email_194@example.com
4    email_592@example.com
Name: email, dtype: object

✅ Profile fields extracted:


In [120]:
# Now drop the profile column
print("\nColumns before dropping profile:")
print(df.columns.tolist())

# Without inplace=True (df remains unchanged)
cleaned_df = df.drop(columns=['profile'])

# With inplace=True (df is modified directly)
#df.drop(columns=['profile'], inplace=True)

print("\nColumns in new DataFrame after dropping profile:")
# print(df.columns.tolist())
print(cleaned_df.columns.tolist())


Columns before dropping profile:
['id', 'name', 'age', 'salary', 'hire_date', 'profile', 'department', 'bonus', 'address', 'phone', 'email']

Columns in new DataFrame after dropping profile:
['id', 'name', 'age', 'salary', 'hire_date', 'department', 'bonus', 'address', 'phone', 'email']


### Step 4: Save cleaned data into new CSV and upload it to S3

In [121]:
print("\n💾 Saving cleaned data to: 'data/cleaned_data.csv' ...")
cleaned_df.to_csv("data/cleaned_data.csv", index=False)
print("✅ Cleaned data saved to: 'data/cleaned_data.csv'")

print(f"\nUploading dataset to s3 bucket: {bucket}")
s3.meta.client.upload_file('data/cleaned_data.csv', bucket, 'output/cleaned_data.csv')
print(f"Dataset 'mock_data.csv' uploaded to: s3://{bucket}/output/cleaned_data.csv")


💾 Saving cleaned data to: 'data/cleaned_data.csv' ...
✅ Cleaned data saved to: 'data/cleaned_data.csv'

Uploading dataset to s3 bucket: sagemaker-us-east-1-891377300841
Dataset 'mock_data.csv' uploaded to: s3://sagemaker-us-east-1-891377300841/output/cleaned_data.csv


## 4. Data Transformation & Feature Engineering

### Step 1: Load the cleaned dataset into new DataFrame

In [122]:
transform_df = pd.read_csv(f's3://{bucket}/output/cleaned_data.csv')
transform_df.head()

,id,name,age,salary,hire_date,department,bonus,address,phone,email
0,1,Name_103,77.0,60000.0,2023-05-11,Marketing,8767.0,"Street 51, City 12",7.323475e+09,email_140@example.com
1,2,Name_436,62.0,50000.0,2021-04-22,Marketing,7553.0,"Street 12, City 6",9.633478e+09,email_38@example.com
2,3,Name_861,61.0,60000.0,2022-04-18,HR,6796.0,NaN,NaN,NaN
3,4,Name_271,36.0,70000.0,NaN,Unknown,8712.0,"Street 17, City 5",8.423076e+09,email_194@example.com
4,5,Name_107,78.0,60000.0,2017-06-17,IT,3340.0,"Street 98, City 37",8.158889e+09,email_592@example.com


### Step 2 : Feature Engineering - Address Length
Create address length feature for potential geographic analysis.

In [123]:
# Create a new column 'address_length' 
print("\n🔧 Creating Address Length Feature...")
transform_df['address_length'] = transform_df['address'].apply(lambda x: len(str(x)))
print("Address followed by Address Length columns")
transform_df[['address', 'address_length']].head()


🔧 Creating Address Length Feature...
Address followed by Address Length columns


,address,address_length
0,"Street 51, City 12",18
1,"Street 12, City 6",17
2,NaN,3
3,"Street 17, City 5",17
4,"Street 98, City 37",18


### Step 3: Feature Engineering - Salary Categorization
Create salary categories for easier analysis and modeling.  
This converts continuous salary into ordinal categories.

In [124]:
print("\n🔧 Creating Salary Categories...")
# Define the bins and labels
bins = [0, 50000, 70000, 100000]
labels = ['low', 'medium', 'high']

# Create a new column 'salary_category'
transform_df['salary_category'] = pd.cut(df['salary'], bins=bins, labels=labels, include_lowest=True)

# Print sample data after adding the 'salary_category' column
print("Sample data after adding the 'salary_category' column: \n")
transform_df[['salary', 'salary_category']].head()


🔧 Creating Salary Categories...
Sample data after adding the 'salary_category' column: 



,salary,salary_category
0,60000.0,medium
1,50000.0,low
2,60000.0,medium
3,70000.0,medium
4,60000.0,medium


### Step 4: Feature Engineering - Age Groups  
Create age groups for demographic analysis.  
This helps in understanding age-based patterns in the data.  

In [125]:
print("\n🔧 Creating Age Groups...")
# Define age bins and labels
age_bins = [0, 25, 35, 45, 55, float('inf')]
age_labels = ['Young', 'Early Career', 'Mid Career', 'Senior', 'Experienced']

# Create a new column 'salary_category'
transform_df['age_group'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, include_lowest=True)

# Age group distribution
print(f"Age group distribution:")
print(transform_df['age_group'].value_counts())

# Print sample data after adding the 'salary_category' column
print("\nSample data after adding the 'age_group' column: \n")
transform_df[['age', 'age_group']].head()



🔧 Creating Age Groups...
Age group distribution:
age_group
Experienced     7318
Senior          4068
Early Career    3142
Mid Career      3022
Young           2450
Name: count, dtype: int64

Sample data after adding the 'age_group' column: 



,age,age_group
0,77.0,Experienced
1,62.0,Experienced
2,61.0,Experienced
3,36.0,Mid Career
4,78.0,Experienced


### Step 5: Feature Engineering - One-Hot Encoding

In [126]:
transform_df = pd.get_dummies(transform_df, columns=['department'], prefix='dept')
print("Top 5 rows with boolean values")
print(transform_df.head())

bool_cols = transform_df.select_dtypes(include='bool').columns
transform_df[bool_cols] = transform_df[bool_cols].astype(int)

print("\nTop 5 rows with numberic values\n")
print(transform_df.head())

Top 5 rows with boolean values
   id      name   age   salary   hire_date   bonus             address  \
0   1  Name_103  77.0  60000.0  2023-05-11  8767.0  Street 51, City 12   
1   2  Name_436  62.0  50000.0  2021-04-22  7553.0   Street 12, City 6   
2   3  Name_861  61.0  60000.0  2022-04-18  6796.0                 NaN   
3   4  Name_271  36.0  70000.0         NaN  8712.0   Street 17, City 5   
4   5  Name_107  78.0  60000.0  2017-06-17  3340.0  Street 98, City 37   

          phone                  email  address_length salary_category  \
0  7.323475e+09  email_140@example.com              18          medium   
1  9.633478e+09   email_38@example.com              17             low   
2           NaN                    NaN               3          medium   
3  8.423076e+09  email_194@example.com              17          medium   
4  8.158889e+09  email_592@example.com              18          medium   

     age_group  dept_Finance  dept_HR  dept_IT  dept_Marketing  dept_Unknown  


### Step 6: Remove missing values in bonus (is the target)

In [127]:
# Check for missing values
print("\n❓ Missing Values Analysis before Removing Missing:\n")
print(transform_df.isnull().sum())

# Remove missing rows for bonus
transform_df = transform_df[transform_df['bonus'].notna()]
print("\n❓ Missing Values Analysis after Removing Missing:\n")
transform_df.isnull().sum()



❓ Missing Values Analysis before Removing Missing:

id                    0
name                  0
age                   0
salary                0
hire_date          2066
bonus              1999
address            2033
phone              2033
email              2033
address_length        0
salary_category       0
age_group             0
dept_Finance          0
dept_HR               0
dept_IT               0
dept_Marketing        0
dept_Unknown          0
dtype: int64

❓ Missing Values Analysis after Removing Missing:



id                    0
name                  0
age                   0
salary                0
hire_date          1863
bonus                 0
address            1808
phone              1808
email              1808
address_length        0
salary_category       0
age_group             0
dept_Finance          0
dept_HR               0
dept_IT               0
dept_Marketing        0
dept_Unknown          0
dtype: int64

### Step 7: Feature Engineering - One-Hot Encoding on age_group & salary_category

In [128]:
transform_df = pd.get_dummies(transform_df, columns=['age_group', 'salary_category'], prefix=['age', 'salary'])
print("Top 5 rows with boolean values")
print(transform_df.head())

bool_cols = transform_df.select_dtypes(include='bool').columns
transform_df[bool_cols] = transform_df[bool_cols].astype(int)

print("\nTop 5 rows with numberic values\n")
print(transform_df.head())

Top 5 rows with boolean values
   id      name   age   salary   hire_date   bonus             address  \
0   1  Name_103  77.0  60000.0  2023-05-11  8767.0  Street 51, City 12   
1   2  Name_436  62.0  50000.0  2021-04-22  7553.0   Street 12, City 6   
2   3  Name_861  61.0  60000.0  2022-04-18  6796.0                 NaN   
3   4  Name_271  36.0  70000.0         NaN  8712.0   Street 17, City 5   
4   5  Name_107  78.0  60000.0  2017-06-17  3340.0  Street 98, City 37   

          phone                  email  address_length  ...  dept_Marketing  \
0  7.323475e+09  email_140@example.com              18  ...               1   
1  9.633478e+09   email_38@example.com              17  ...               1   
2           NaN                    NaN               3  ...               0   
3  8.423076e+09  email_194@example.com              17  ...               0   
4  8.158889e+09  email_592@example.com              18  ...               0   

   dept_Unknown  age_Young  age_Early Career  age

### Step 8: Feature Engineering | Calculate Tenure

In [129]:
print("Convert hire_date to datetime")
transform_df['hire_date'] = pd.to_datetime(transform_df['hire_date'], errors='coerce')
print(transform_df['hire_date'].head())
# print(transform_df.dtypes)

# non_date_rows = transform_df[transform_df['hire_date'].apply(lambda x: isinstance(x, str))]
# print("non date rows:")
# print(non_date_rows)

print("Calculate Tenure in Days....")
transform_df['tenure_days'] = (pd.Timestamp('now') - transform_df['hire_date']).dt.days

print("Calculated Tenure Days")
print(transform_df['tenure_days'])

print("Handle Missing values of tenure_days")
transform_df['tenure_days'] = transform_df['tenure_days'].fillna(transform_df['tenure_days'].median())

print("Tenure Days after handled missing days")
print(transform_df['tenure_days'])

Convert hire_date to datetime
0   2023-05-11
1   2021-04-22
2   2022-04-18
3          NaT
4   2017-06-17
Name: hire_date, dtype: datetime64[ns]
Calculate Tenure in Days....
Calculated Tenure Days
0         808.0
1        1557.0
2        1196.0
3           NaN
4        2962.0
          ...  
19994     549.0
19996    3570.0
19997    3498.0
19998     253.0
19999    1097.0
Name: tenure_days, Length: 18001, dtype: float64
Handle Missing values of tenure_days
Tenure Days after handled missing days
0         808.0
1        1557.0
2        1196.0
3        1834.0
4        2962.0
          ...  
19994     549.0
19996    3570.0
19997    3498.0
19998     253.0
19999    1097.0
Name: tenure_days, Length: 18001, dtype: float64


### Step 9: Feature Engineering | Removing Irrelevant or non-predictive columns


In [130]:
print("Dropping ID, Address, Phone, Name, Hire Date, and Email.....")
transform_df.drop(columns=['id', 'address', 'phone', 'email', 'name', 'hire_date'], inplace=True)
print("After dropping ID, Address, Phone, Name, Hire Date, and Email, dataset look like")
print(transform_df.head())

Dropping ID, Address, Phone, Name, Hire Date, and Email.....
After dropping ID, Address, Phone, Name, Hire Date, and Email, dataset look like
    age   salary   bonus  address_length  dept_Finance  dept_HR  dept_IT  \
0  77.0  60000.0  8767.0              18             0        0        0   
1  62.0  50000.0  7553.0              17             0        0        0   
2  61.0  60000.0  6796.0               3             0        1        0   
3  36.0  70000.0  8712.0              17             0        0        0   
4  78.0  60000.0  3340.0              18             0        0        1   

   dept_Marketing  dept_Unknown  age_Young  age_Early Career  age_Mid Career  \
0               1             0          0                 0               0   
1               1             0          0                 0               0   
2               0             0          0                 0               0   
3               0             1          0                 0               1   
4

### Step 7: Save the transformed DataFrame to a new csv file

In [131]:
print("Saving Transformed data csv to: 'data/transformed_data.csv' ...")
transform_df.to_csv("data/transformed_data.csv", index=False)
print("\nTransformed data csv saved to: 'data/transformed_data.csv'")

s3.meta.client.upload_file('data/transformed_data.csv', bucket, 'output/transformed_data.csv')
print(f"Transformed data 'transformed_data.csv' uploaded to: s3://{bucket}/output/transformed_data.csv")

Saving Transformed data csv to: 'data/transformed_data.csv' ...

Transformed data csv saved to: 'data/transformed_data.csv'
Transformed data 'transformed_data.csv' uploaded to: s3://sagemaker-us-east-1-891377300841/output/transformed_data.csv
